<a href="https://colab.research.google.com/github/AnSe2002/Project-workshop-on-the-development-of-ETL-solutions/blob/main/vebinar_28_03_2025/vebinar_28_03_2025_Sergeeva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загрузка файлов.

In [1]:
from google.colab import files
files.upload()

Saving stats_2025-03-28.csv to stats_2025-03-28.csv
Saving download_report_2025-03-28.csv to download_report_2025-03-28.csv
Saving summary_report.csv to summary_report.csv


{'stats_2025-03-28.csv': b'total,success,failed,start_time\r\n10,10,0,2025-03-28T12:15:40.300247\r\n',
 'download_report_2025-03-28.csv': b'url,status,error,filename,timestamp\r\nhttps://thespacedevs-prod.nyc3.digitaloceanspaces.com/media/images/spectrum_on_the_image_20250321072643.jpeg,success,,spectrum_on_the_image_20250321072643.jpeg,2025-03-28T12:15:40.300577\r\nhttps://thespacedevs-prod.nyc3.digitaloceanspaces.com/media/images/long_march_7a_image_20210311201838.jpg,success,,long_march_7a_image_20210311201838.jpg,2025-03-28T12:15:45.108452\r\nhttps://thespacedevs-prod.nyc3.digitaloceanspaces.com/media/images/firefly_alpha_l_image_20240605174156.jpeg,success,,firefly_alpha_l_image_20240605174156.jpeg,2025-03-28T12:15:45.995387\r\nhttps://thespacedevs-prod.nyc3.digitaloceanspaces.com/media/images/falcon2520925_image_20221009234147.png,success,,falcon2520925_image_20221009234147.png,2025-03-28T12:15:46.776985\r\nhttps://thespacedevs-prod.nyc3.digitaloceanspaces.com/media/images/eris_o

In [3]:
import pandas as pd
df = pd.read_csv('stats_2025-03-28.csv')
df.head()

,total,success,failed,start_time
0,10,10,0,2025-03-28T12:15:40.300247


Всего было загружено 10 изображений, все успешные, дата запуска 28 марта.

In [6]:
df_2 = pd.read_csv('download_report_2025-03-28.csv')
df_2.head()

,url,status,error,filename,timestamp
0,https://thespacedevs-prod.nyc3.digitaloceanspa...,success,NaN,spectrum_on_the_image_20250321072643.jpeg,2025-03-28T12:15:40.300577
1,https://thespacedevs-prod.nyc3.digitaloceanspa...,success,NaN,long_march_7a_image_20210311201838.jpg,2025-03-28T12:15:45.108452
2,https://thespacedevs-prod.nyc3.digitaloceanspa...,success,NaN,firefly_alpha_l_image_20240605174156.jpeg,2025-03-28T12:15:45.995387
3,https://thespacedevs-prod.nyc3.digitaloceanspa...,success,NaN,falcon2520925_image_20221009234147.png,2025-03-28T12:15:46.776985
4,https://thespacedevs-prod.nyc3.digitaloceanspa...,success,NaN,eris_on_the_lau_image_20250227073032.jpg,2025-03-28T12:15:47.649213


Показан URL изображения, статус загрузки и имя файла.

In [7]:
df_3 = pd.read_csv('summary_report.csv')
df_3.head()

,date,total,success,failed,start_time
0,2025-03-28,10,10,0,2025-03-28T12:15:33.988395
1,2025-03-28,10,10,0,2025-03-28T12:15:40.300247


Запуск выполнялся 2 раза за сегодня по историческим данным и успешно.

Предварительная обработка.

In [8]:
df_2['timestamp'] = pd.to_datetime(df_2['timestamp'])
df['start_time'] = pd.to_datetime(df['start_time'])
df_3['start_time'] = pd.to_datetime(df_3['start_time'])

Анализ дубликатов.

In [9]:
duplicates = df_2[df_2.duplicated('filename', keep=False)]
if not duplicates.empty:
    print("Обнаружены дубликаты файлов:")
    print(duplicates[['filename', 'url']])
else:
    print("Дубликаты не обнаружены")

Обнаружены дубликаты файлов:
                                 filename  \
3  falcon2520925_image_20221009234147.png   
5  falcon2520925_image_20221009234147.png   

                                                 url  
3  https://thespacedevs-prod.nyc3.digitaloceanspa...  
5  https://thespacedevs-prod.nyc3.digitaloceanspa...  


Всего выявлено 2 дубликата.

Временные характеристики загрузки.

In [12]:
import altair as alt

# Подготовка данных
df_2 = df_2.sort_values('timestamp')
df_2['download_time'] = range(1, len(df_2)+1)

In [13]:
# Построение графика
chart = alt.Chart(df_2).mark_line(point=True).encode(
    x=alt.X('timestamp:T', title='Время', axis=alt.Axis(format='%H:%M:%S')),
    y=alt.Y('download_time:Q', title='Количество загруженных файлов'),
    tooltip=['timestamp', 'download_time']
).properties(
    title=f'Прогресс загрузки ({len(df_2)} файлов)',
    width=800,
    height=400
)

# Добавление меток
text = chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-10  # Смещение метки выше точки
).encode(
    text='download_time:Q'
)

# Комбинирование и отображение
(chart + text).configure_axis(
    labelAngle=45
).configure_view(
    strokeWidth=0
).interactive()

alt.LayerChart(...)

Файлы были загружены за 11 секунд, линия растёт равномерно, значит, загрузка стабильная со 2-ого файла, при этом разница между 1 и 2 файлом равна 4 секундам.

Анализ временных задержек между загрузками.

In [14]:
df_2['time_diff'] = df_2['timestamp'].diff().dt.total_seconds()
print(f"Среднее время между загрузками: {df_2['time_diff'].mean():.2f} сек")

Среднее время между загрузками: 1.25 сек


Файлы загружаются в среднем быстро, за 1 секунду, что говорит о высокой производителньости DAG.